In [ ]:
import re
import pandas as pd
from rapidfuzz import process, fuzz

# -----------------------------------------
# 1. Paths
# -----------------------------------------
hq_path  = "hqontario_ed_all_metrics.csv"
geo_path = "Ministry_of_Health_Hospitals_Geo.csv"

hq  = pd.read_csv(hq_path)   # has HOSPITAL + ED metrics
geo = pd.read_csv(geo_path)  # has ENGLISH_NAME, LONGITUDE, LATITUDE, ADDRESS, SERVICE_TYPE, ...

print("HQO hospitals:", len(hq))
print("MOH locations:", len(geo))

# -----------------------------------------
# 2. Normalization helper
# -----------------------------------------
def normalize(name: str) -> str:
    if not isinstance(name, str):
        return ""
    s = name.upper()
    s = re.sub(r"[^A-Z0-9]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

hq["norm"]  = hq["HOSPITAL"].apply(normalize)
geo["norm"] = geo["ENGLISH_NAME"].apply(normalize)

# -----------------------------------------
# 3. Manual override mapping (17 tricky ones)
#    KEY:  HOSPITAL (from HQ file)
#    VAL:  ENGLISH_NAME (from MOH geo file)
# -----------------------------------------
manual_map = {
    "Mackenzie Health-Jane Street Site":
        "MacKenzie Health - Cortellucci Vaughan Hospital",

    "Brightshores Health System-Lions Head Site":
        "Grey Bruce Health Services - Lion's Head",

    "Brightshores Health System-Meaford Site":
        "Grey Bruce Health Services - Meaford",

    "St. Joseph's Health Care System-Hamilton":
        "St. Joseph's Healthcare Hamilton - Charlton Campus",

    "Bluewater Health-Charlotte Eleanor Englehart (Petrolia)":
        "Bluewater Health - Petrolia",

    "Scarborough Health Network-Scar.Gen.Site":
        "Scarborough Health Network - Scarborough General",

    "Brightshores Health System-Markdale Site":
        "Grey Bruce Health Services - Markdale",

    "Brightshores Health System-Wiarton Site":
        "Grey Bruce Health Services - Wiarton",

    # Sinai Health System: MOH dataset only has Hennick Bridgepoint,
    # so we map to that to at least get a Toronto coordinate.
    "Sinai Health System-Mount Sinai Site":
        "Hennick Bridgepoint Hospital",

    "Quinte Health - Bancroft":
        "Quinte Health - North Hastings",

    "Brightshores Health System-Southampton Site":
        "Grey Bruce Health Services - Southampton",

    "University Health Network-General Site":
        "University Health Network - Toronto General",

    "Health Sciences North-Laurentian":
        "Health Sciences North - Ramsay Lake Health Centre",

    "Perth & Smiths Falls Dist-Smiths Falls Site":
        "Perth and Smiths Falls District Hospital - Smiths Falls",

    "Brightshores Health System-Owen Sound":
        "Grey Bruce Health Services - Owen Sound",

    "Grand River Hospital Corp-Waterloo Site":
        "Grand River Hospital - Kitchener-Waterloo",

    "Sault Area Hospital-Sault Ste. Marie":
        "Sault Area Hospital - General Site",
}

# sanity check: all manual keys exist in HQ
missing_manual_keys = set(manual_map.keys()) - set(hq["HOSPITAL"])
if missing_manual_keys:
    print("Warning: these manual_map keys not found in HQ file:", missing_manual_keys)

# -----------------------------------------
# 4. Build full merge: manual first, fuzzy fallback
# -----------------------------------------
def build_full_merge(hq_df, geo_df, manual):
    hq_df = hq_df.copy()
    geo_df = geo_df.copy()

    # Ensure norm columns exist
    if "norm" not in hq_df:
        hq_df["norm"] = hq_df["HOSPITAL"].apply(normalize)
    if "norm" not in geo_df:
        geo_df["norm"] = geo_df["ENGLISH_NAME"].apply(normalize)

    merged_rows = []

    geo_norms = geo_df["norm"].tolist()

    for _, row in hq_df.iterrows():
        hosp = row["HOSPITAL"]

        # ----- Case 1: manual override -----
        if hosp in manual:
            target_name = manual[hosp]
            matches_idx = geo_df[geo_df["ENGLISH_NAME"] == target_name].index
            if len(matches_idx) == 0:
                print("Manual target not found in geo file:", hosp, "->", target_name)
                continue
            g = geo_df.loc[matches_idx[0]]
            merged_rows.append({
                **row.to_dict(),
                "ENGLISH_NAME": g["ENGLISH_NAME"],
                "LONGITUDE":    g["LONGITUDE"],
                "LATITUDE":     g["LATITUDE"],
                "ADDRESS":      g["ADDRESS"],
                "MATCH_METHOD": "manual",
                "MATCH_SCORE":  None,
            })
        else:
            # ----- Case 2: fuzzy auto-match -----
            h_norm = row["norm"]
            best = process.extractOne(h_norm, geo_norms, scorer=fuzz.WRatio)
            if best is None:
                print("No fuzzy match found for:", hosp)
                continue

            g_norm, score, idx_geo = best
            g = geo_df.iloc[idx_geo]

            merged_rows.append({
                **row.to_dict(),
                "ENGLISH_NAME": g["ENGLISH_NAME"],
                "LONGITUDE":    g["LONGITUDE"],
                "LATITUDE":     g["LATITUDE"],
                "ADDRESS":      g["ADDRESS"],
                "MATCH_METHOD": "auto",
                "MATCH_SCORE":  score,
            })

    return pd.DataFrame(merged_rows)

merged = build_full_merge(hq, geo, manual_map)

print("Merged rows:", len(merged))
print(merged["MATCH_METHOD"].value_counts())

# -----------------------------------------
# 5. Save final merged dataset
# -----------------------------------------
merged.to_csv("hospital_ed_merged_with_geo.csv", index=False)
print("Saved hospital_ed_merged_with_geo.csv")





HQO hospitals: 126
MOH locations: 231
Merged rows: 126
MATCH_METHOD
auto      109
manual     17
Name: count, dtype: int64
Saved hospital_ed_merged_with_geo.csv


Drop Redundant ENGLISH_NAME column

In [7]:
merged_path = "hospital_ed_merged_with_geo.csv"
df = pd.read_csv(merged_path)

# Drop ENGLISH_NAME to avoid redundancy
df = df.drop(columns=["ENGLISH_NAME"])

# Option 1: overwrite the same file
df.to_csv("hospital_ed_merged_with_geo.csv", index=False)